In [100]:
from bs4 import BeautifulSoup
import requests
import nltk
import math
nltk.download('stopwords')
import pandas as pd
import pickle
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import os
import networkx as nx
import networkx.algorithms.community as nx_community
from nltk.corpus import stopwords
import re
import string
from nltk.tokenize import word_tokenize
import networkx.algorithms.community as nx_comm
import netwulf
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
from collections import Counter


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Webscraping 
Retrieving all character names from wikisite: 
"https://en.wikipedia.org/wiki/List_of_Harry_Potter_characters"

In [35]:
# Characterlist from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_Harry_Potter_characters'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# List for characters
character_list = []
content_div = soup.find('div', {'id': 'bodyContent'})
for heading in content_div.find_all('h3'):
    # find the <ul> tag immediately following this heading
    character_ul = heading.find_next_sibling('ul')
    for character_li in character_ul.find_all('li'):
        # extract the character name from the <a> tag within the <li>, if it exists
        character_a = character_li.find('a')
        if character_a:
            character_name = character_a.text
            character_list.append(character_name)

print(character_list)

['Hannah Abbott', 'Ludo Bagman', 'Bathilda Bagshot', 'Katie Bell', 'Cuthbert Binns', 'Phineas Nigellus Black', 'Sirius Black', 'Amelia Bones', 'Susan Bones', 'Terry Boot', 'Lavender Brown', 'Slytherin', 'Charity Burbage', 'Frank Bryce', 'Alecto Carrow', 'Amycus Carrow', 'Reginald Cattermole', 'Cho Chang', 'Percy Weasley', 'Crabbe', 'Vincent Crabbe', 'Colin Creevey', 'Dennis Creevey', 'Dirk Cresswell', 'Barty Crouch Sr', 'Barty Crouch Jr', 'John Dawlish', 'Fleur Delacour', 'Dedalus Diggle', 'Amos Diggory', 'Cedric Diggory', 'Elphias Doge', 'Antonin Dolohov', 'Aberforth Dumbledore', 'Albus Dumbledore', 'Ariana Dumbledore', 'Dudley Dursley', 'Marge Dursley', 'Petunia Dursley', 'Vernon Dursley', 'Marietta Edgecombe', 'Arabella Figg', 'Argus Filch', 'Justin Finch-Fletchley', 'Seamus Finnigan', 'Mundungus Fletcher', 'Filius Flitwick', 'Cornelius Fudge', 'Marvolo Gaunt', 'Merope Gaunt', 'Morfin Gaunt', 'Anthony Goldstein', 'Goyle', 'Gregory Goyle', 'Hermione Granger', 'Draco Malfoy', 'Eastern

In [36]:
# Ikke optimal løsning, men fikser alle fejlene, så vi kan smide en liste til API'en lidt hurtigt. 
replacements = {
    "Slytherin": "Millicent Bulstrode",
    "Percy Weasley": "Penelope Clearwater",
    "Draco Malfoy": "Astoria Greengrass",
    "Eastern European": "Gregorovitch",
    "Diagon Alley": "Madam Malkin",
    "Horace Slughorn": "Cormac McLaggen",
    "Vanishing Cabinet": "Graham Montague",
    "Harry Potter and the Cursed Child": "Delphi Riddle",
    "Merope Gaunt": "Tom Riddle Sr.",
    "Lord Voldemort": "Thomas Marvolo Riddle",
    "Bogrod": "Gringotts",
    "Gringotts": "Gornuk",
    "Black family": "Kreacher",
    "Death Eater" : "Travers",
    "Barty Crouch Sr": "Winky"}

for i in range(len(character_list)):
    if character_list[i] in replacements:
        character_list[i] = replacements[character_list[i]]

additions = ["Gabrielle Delacour", 
             "Kendra Dumbledore", 
             "Percival Dumbledore", 
             "Nicolas Flamel", 
             "Griselda Marchbanks", 
             "Eloise Midgen", 
             "Theodore Nott", 
             "Nott Sr", 
             "Albus Severus Potter", 
             "James Sirius Potter", 
             "Lily Luna Potter", 
             "Mary Riddle", 
             "Thomas Riddle", 
             "Demelza Robins", 
             "Salazar Slytherin", 
             "Romilda Vane", 
             "Hugo Weasley", 
             "Rose Weasley",
             "Millicent Bulstrode",
             "Blaise Zabini",
             "Sir Cadogan",
             "Scabior",
             "Trevor"]

print(character_list)

['Hannah Abbott', 'Ludo Bagman', 'Bathilda Bagshot', 'Katie Bell', 'Cuthbert Binns', 'Phineas Nigellus Black', 'Sirius Black', 'Amelia Bones', 'Susan Bones', 'Terry Boot', 'Lavender Brown', 'Millicent Bulstrode', 'Charity Burbage', 'Frank Bryce', 'Alecto Carrow', 'Amycus Carrow', 'Reginald Cattermole', 'Cho Chang', 'Penelope Clearwater', 'Crabbe', 'Vincent Crabbe', 'Colin Creevey', 'Dennis Creevey', 'Dirk Cresswell', 'Winky', 'Barty Crouch Jr', 'John Dawlish', 'Fleur Delacour', 'Dedalus Diggle', 'Amos Diggory', 'Cedric Diggory', 'Elphias Doge', 'Antonin Dolohov', 'Aberforth Dumbledore', 'Albus Dumbledore', 'Ariana Dumbledore', 'Dudley Dursley', 'Marge Dursley', 'Petunia Dursley', 'Vernon Dursley', 'Marietta Edgecombe', 'Arabella Figg', 'Argus Filch', 'Justin Finch-Fletchley', 'Seamus Finnigan', 'Mundungus Fletcher', 'Filius Flitwick', 'Cornelius Fudge', 'Marvolo Gaunt', 'Tom Riddle Sr.', 'Morfin Gaunt', 'Anthony Goldstein', 'Goyle', 'Gregory Goyle', 'Hermione Granger', 'Astoria Greengr

181

## Collecting data with Harry Potter API

In [4]:
# API for facts about Harry Potter characters
API = {"data":{"message":"Welcome to the Potter DB API!","api_routes":["/v1/books/:book_id/chapters","/v1/books/:book_id/chapters/:id","/v1/books","/v1/books/:id","/v1/characters","/v1/characters/:id","/v1/movies","/v1/movies/:id","/v1/potions","/v1/potions/:id","/v1/spells","/v1/spells/:id"]}}
API

{'data': {'message': 'Welcome to the Potter DB API!',
  'api_routes': ['/v1/books/:book_id/chapters',
   '/v1/books/:book_id/chapters/:id',
   '/v1/books',
   '/v1/books/:id',
   '/v1/characters',
   '/v1/characters/:id',
   '/v1/movies',
   '/v1/movies/:id',
   '/v1/potions',
   '/v1/potions/:id',
   '/v1/spells',
   '/v1/spells/:id']}}

In [5]:
# API link
BASE_URL = "https://api.potterdb.com/"
VERSION = "v1/"
RESOURCE = "characters"

my_url = BASE_URL + VERSION + RESOURCE
print(my_url)

https://api.potterdb.com/v1/characters


In [6]:
# Loading data
params = {'query': '{}'.format(character_list), "offset":0,
           "limit":1}
r = requests.get(my_url, params=params)
data = r.json()["data"]

# Setting all names from characterlist as keys in dictionary holding attributes
all_data = {}
for i in range(len(character_list)):
    all_data[character_list[i]] = None

# Comparing names from data with characterlist
for i in data:
    name = (i['attributes']['name'])
    for j in all_data.keys():
        #print(j)
        if name == j:
            print('yes')
            print(name)
            all_data[j] = i['attributes']['blood_status']

#print(all_data['blood_status'])

yes
Aberforth Dumbledore
yes
Albert Runcorn
yes
Alecto Carrow


In [75]:
# API link
BASE_URL = "https://api.potterdb.com/"
VERSION = "v1/"
RESOURCE = "characters"

my_url = BASE_URL + VERSION + RESOURCE
print(my_url)
all_data = {}
counter = 0
for i in range(50):

    url = my_url + f"?page[number={i+1}]&page[size=100]"
    r = requests.get(url)
    data = r.json()["data"]

    for i in data:
        all_data[counter] = {}
        name = (i['attributes']['name'])
        all_data[counter]['Name'] = i['attributes']['name']
        all_data[counter]['Blood status'] = i['attributes']['blood_status']
        all_data[counter]['House'] = i['attributes']['house']
        all_data[counter]['Species'] = i['attributes']['species']
        all_data[counter]['Death time'] = i['attributes']['died']
        all_data[counter]['Alias'] = i['attributes']['alias_names']
        all_data[counter]['Wiki'] = i['attributes']['wiki']
        all_data[counter]['Gender'] = i['attributes']['gender']
        counter +=1
    
df_data = pd.DataFrame.from_dict(all_data, orient='index')

https://api.potterdb.com/v1/characters


In [76]:
df_data

,name,Blood status,House,Species,Death time,Alias,Wiki,Gender
0,1992 Gryffindor vs Slytherin Quidditch match s...,None,None,None,None,None,https://harrypotter.fandom.com/wiki/1992_Gryff...,None
1,1996 Gryffindor Quidditch Keeper trials specta...,None,None,None,None,None,https://harrypotter.fandom.com/wiki/1996_Gryff...,None
2,19th-century Flying Instructor,None,None,Human,None,None,https://harrypotter.fandom.com/wiki/Unidentifi...,Female
3,19th-century Herbology professor,None,None,Human,None,None,https://harrypotter.fandom.com/wiki/Unidentifi...,Female
4,19th-century Potions professor,None,None,Human,None,None,https://harrypotter.fandom.com/wiki/Unidentifi...,Male
...,...,...,...,...,...,...,...,...
4061,Zoo director,Muggle,None,Human,None,None,https://harrypotter.fandom.com/wiki/Zoo_director,Male
4062,Zubeida Khan,None,Slytherin,Human,None,None,https://harrypotter.fandom.com/wiki/Zubeida_Khan,Female
4063,Zygmunt Budge,None,None,Human,None,"['Fungus-face' (by his mother), The Hermit of ...",https://harrypotter.fandom.com/wiki/Zygmunt_Budge,Male
4064,Zygmunt Budge's great-grandmother,None,None,Human,None,None,https://harrypotter.fandom.com/wiki/Zygmunt_Bu...,Female


In [78]:
# Filter data after having some information of Blood status, house, species, death time or alias
df_data[~((df_data["Blood status"].isnull())&(df_data["House"].isnull())&(df_data["Species"].isnull())&(df_data["Death time"].isnull())&(df_data["Alias"].isnull()))]

,name,Blood status,House,Species,Death time,Alias,Wiki,Gender
2,19th-century Flying Instructor,None,None,Human,None,None,https://harrypotter.fandom.com/wiki/Unidentifi...,Female
3,19th-century Herbology professor,None,None,Human,None,None,https://harrypotter.fandom.com/wiki/Unidentifi...,Female
4,19th-century Potions professor,None,None,Human,None,None,https://harrypotter.fandom.com/wiki/Unidentifi...,Male
5,2-Headed Baby,None,None,Human,None,None,https://harrypotter.fandom.com/wiki/2-Headed_Baby,Male (most likely)
6,8-year-old Muggle girl,Muggle,None,Human,None,None,https://harrypotter.fandom.com/wiki/Unidentifi...,Female
...,...,...,...,...,...,...,...,...
4061,Zoo director,Muggle,None,Human,None,None,https://harrypotter.fandom.com/wiki/Zoo_director,Male
4062,Zubeida Khan,None,Slytherin,Human,None,None,https://harrypotter.fandom.com/wiki/Zubeida_Khan,Female
4063,Zygmunt Budge,None,None,Human,None,"['Fungus-face' (by his mother), The Hermit of ...",https://harrypotter.fandom.com/wiki/Zygmunt_Budge,Male
4064,Zygmunt Budge's great-grandmother,None,None,Human,None,None,https://harrypotter.fandom.com/wiki/Zygmunt_Bu...,Female


## Webscrape text data and node links from wiki

In [114]:
# Function to webscrape Harry Potter wiki for text and links
def webscrapeWiki(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    content_div = soup.find('div', {'id': 'mw-content-text'})
    text = ""
    for p in content_div.find_all('p'):
        text += p.get_text() + " "

    links = []
    for a in content_div.find_all("a"):
        split = a['href'].split("/")
        if len(split)>1:
            if split[1]=="wiki":
                links.append(split[2])

    return text, links

df_data["Wiki text"], df_data["Character links"] = zip(*df_data["Wiki"].apply(webscrapeWiki))

In [123]:
df_data

,name,Blood status,House,Species,Death time,Alias,Wiki,Gender,Wiki text,Character links,Wiki name,Sum character links
0,1992 Gryffindor vs Slytherin Quidditch match s...,None,None,None,None,None,https://harrypotter.fandom.com/wiki/1992_Gryff...,None,The title of this article is conjectural. Alt...,"[Canon, Filius_Flitwick, Irma_Pince, Severus_S...",1992_Gryffindor_vs_Slytherin_Quidditch_match_s...,"{'Filius_Flitwick': 2, 'Irma_Pince': 3, 'Sever..."
1,1996 Gryffindor Quidditch Keeper trials specta...,None,None,None,None,None,https://harrypotter.fandom.com/wiki/1996_Gryff...,None,"In September 1996, a number of unidentified sp...","[Harry_Potter_and_the_Half-Blood_Prince, Septe...",1996_Gryffindor_Quidditch_Keeper_trials_specta...,"{'Harry_Potter': 1, 'Ronald_Weasley': 1, 'Corm..."
2,19th-century Flying Instructor,None,None,Human,None,None,https://harrypotter.fandom.com/wiki/Unidentifi...,Female,At least some content in this article is deriv...,"[Hogwarts_Legacy, 1872, Japan, Madam, Human, C...",Unidentified_19th-century_Flying_Instructor,{'Unidentified_19th-century_Hogwarts_student':...
3,19th-century Herbology professor,None,None,Human,None,None,https://harrypotter.fandom.com/wiki/Unidentifi...,Female,At least some content in this article is deriv...,"[Hogwarts_Legacy, 1873, London, England, Great...",Unidentified_19th-century_Herbology_professor,{'Unidentified_19th-century_Hogwarts_student':...
4,19th-century Potions professor,None,None,Human,None,None,https://harrypotter.fandom.com/wiki/Unidentifi...,Male,At least some content in this article is deriv...,"[Hogwarts_Legacy, 1850s, Great_Britain, Irelan...",Unidentified_19th-century_Potions_professor,{'Unidentified_19th-century_Hogwarts_student':...
...,...,...,...,...,...,...,...,...,...,...,...,...
4061,Zoo director,Muggle,None,Human,None,None,https://harrypotter.fandom.com/wiki/Zoo_director,Male,The title of this article is conjectural. Alt...,"[Canon, Non-magic_people, Great_Britain, Human...",Zoo_director,{'Petunia_Dursley': 1}
4062,Zubeida Khan,None,Slytherin,Human,None,None,https://harrypotter.fandom.com/wiki/Zubeida_Khan,Female,\n\nZubeida Khan\n\n\nBiographical information...,"[1_September, 1975, 31_August, 1982, Great_Bri...",Zubeida_Khan,{}
4063,Zygmunt Budge,None,None,Human,None,"['Fungus-face' (by his mother), The Hermit of ...",https://harrypotter.fandom.com/wiki/Zygmunt_Budge,Male,At least some content in this article is deriv...,"[Harry_Potter:_Magic_Awakened, 1490, 1491, 149...",Zygmunt_Budge,"{'Arsenius_Jigger': 1, 'Bilius_Finbok': 1, 'Bl..."
4064,Zygmunt Budge's great-grandmother,None,None,Human,None,None,https://harrypotter.fandom.com/wiki/Zygmunt_Bu...,Female,The title of this article is conjectural. Alt...,"[Canon, Human, Zygmunt_Budge%27s_mother, Zygmu...",Zygmunt_Budge's_great-grandmother,{'Zygmunt_Budge': 4}


In [120]:
def wikiName(wikiURL):
    return wikiURL.split("/")[-1]

def checkLinks(linkCounter, wikiNames):
    cleanLinks = linkCounter.copy()
    for link in linkCounter.keys():
        if link not in list(wikiNames):
            del cleanLinks[link]
    return cleanLinks

df_data["Wiki name"] = df_data["Wiki"].apply(wikiName)
df_data["Sum character links"] = df_data["Character links"].apply(Counter)
df_data["Sum character links"] = df_data["Sum character links"].apply(checkLinks,wikiNames=df_data["Wiki name"])


In [124]:
#Save dataframe
df_data.to_pickle("df_data")


In [125]:
# Load dataframe:
df_data_loaded = pd.read_pickle("df_data")

## Code snippet for webscraping of actual wikipedia

In [25]:
for name in character_list:
    name_url = name.replace(' ', '_')
    url = f"https://en.wikipedia.org/wiki/{name_url}"

    # Send request 
    response = requests.get(url)

    # Exists? 
    if response.status_code == 200:
        print(f"{name}: Wikipedia page exists.")
    else:
        print(f"{name}: Wikipedia page does not exist.")